<a href="https://colab.research.google.com/github/cjfghk5697/Practice_Pytorch/blob/main/Practice_Pytorch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [4]:
#dataset 클래스로 만듦 train dataset

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True, #train=true는 FashionMNIST라는 곳에서 train dataset을 가져왔다는 뜻.
    download=True, #만약에 download를 안했으면 download해라
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False, #train=False로 test 데이터 가져옴
    download=True,
    transform=ToTensor(),
)


In [7]:
batch_size=64

#create data loaders.
train_dataloader= DataLoader(training_data, batch_size=batch_size)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print("Shape of X[N, C, H, W]:",X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


training데이터는 데이터 자체만 있음. batch_size는 몇개 할 건지(미니배치학습) DATALOADER는 데이터 환경 조성

N C H W는 넘버 체널 하잇트 웨이트
채널 수, 어떤 채널, 세로, 가로 크기

채널은 1이면 그레이. 3이면 RGB
X는 64장이 있다. 64*28*28사이즈라고 보면된다.
y는 데이터들에 대한 레이블이다.

In [8]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() #부모속성 nn.module을 초기화 
        self.flatten = nn.Flatten() 
        self.linear_relu_stack = nn.Sequential( #시퀀서는 순서대로 라는 뜻
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10), #10개의 클래스가 저장됨
            nn.ReLU()
        )

    def forward(self, x): #사용
        x = self.flatten(x) #nn.Module에 이미 정의 되어있음. 재정의 함.
        
        logits = self.linear_relu_stack(x) #x를 계싼
        return logits #prediction한 

model = NeuralNetwork().to(device) #cpu로 보낸 거임. cpu로 올리겠다.
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


cuda가 사용이 가능하면 cuda를 쓰겠다. using cpu가 print됨 즉 cpu를 씀.

nn.Module

In [14]:
loss_fn = nn.CrossEntropyLoss() #척도
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) #최적화된 것. 기울기로 보고 찾아감 loss가 작아지는 기울기


In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader): #batch도 가져옴
        X, y = X.to(device), y.to(device) #cpu에 보냄

        # Compute prediction error
        pred = model(X) #forward 함수가 실행됨. 이게 nn.Module에 자동으로 구현 되어있음. logits이 prediction한 값
        loss = loss_fn(pred, y) #얼마나 정확한지 방향 정하기


        # Backpropagation
        optimizer.zero_grad() #gradient를 초기화해라. 0으로 만들어라 어떻게 등산했는지 까먹어라 길을 다시 정하겠다.
        loss.backward() #그 방향으로 내려가기. 업데이트 함. optimizer는 SGD씀
        optimizer.step() #얼마나 내려왔는지 알려줌 어떻게 내려왔는지도 알려줌

        if batch % 100 == 0: #100번에 한번씩 PRINT해라
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
def test(dataloader, model, loss_fn): #여기서는 학습 x 테스트만
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): #우리는 학습하지 않겠다?
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() #prad.argmax(1)은 첫번째로 가장큰 예측값의 index 줌. index가 y랑 같으면 true 줌. 그럼 type을 float으로 바꿈. 다 같으면 sum에서 30이 나올것임 coorrect는 계속 더해서 맞은 값을 보여줌
    test_loss /= num_batches
    correct /= size #맞을 확률
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296002  [    0/60000]
loss: 2.287408  [ 6400/60000]
loss: 2.282100  [12800/60000]
loss: 2.282075  [19200/60000]
loss: 2.245959  [25600/60000]
loss: 2.256192  [32000/60000]
loss: 2.244469  [38400/60000]
loss: 2.233786  [44800/60000]
loss: 2.236921  [51200/60000]
loss: 2.186704  [57600/60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 2.203479 

Epoch 2
-------------------------------
loss: 2.227092  [    0/60000]
loss: 2.202825  [ 6400/60000]
loss: 2.190003  [12800/60000]
loss: 2.182369  [19200/60000]
loss: 2.102553  [25600/60000]
loss: 2.143195  [32000/60000]
loss: 2.119778  [38400/60000]
loss: 2.111968  [44800/60000]
loss: 2.124761  [51200/60000]
loss: 2.003280  [57600/60000]
Test Error: 
 Accuracy: 46.4%, Avg loss: 2.048743 

Epoch 3
-------------------------------
loss: 2.121543  [    0/60000]
loss: 2.056239  [ 6400/60000]
loss: 2.036685  [12800/60000]
loss: 2.013618  [19200/60000]
loss: 1.855148  [25600/60000]
loss: 1.971614  [32000/600